In [1]:
import pandas as pd
from pathlib import Path

In [2]:
data_root = Path("../data")
raw_data_root = data_root / "raw_data"

In [11]:
arena_raw_df = pd.read_parquet(raw_data_root / "search-arena-chat-24k.parquet")

In [33]:
cleaned_arena_data_root = data_root / "intermediate" / "cleaned_arena_data"

threads_df = pd.read_parquet(cleaned_arena_data_root / "threads.parquet")
questions_df = pd.read_parquet(cleaned_arena_data_root / "questions.parquet")
responses_df = pd.read_parquet(cleaned_arena_data_root / "responses.parquet")
citations_df = pd.read_parquet(cleaned_arena_data_root / "citations.parquet")

In [6]:
model_response_mapping = dict(responses_df.model_name.value_counts())

In [7]:
model_name_mapping = {
    "gpt-4o-mini-search-preview": "gpt-4o-mini-search",
    "gpt-4o-mini-search-preview-high": "gpt-4o-mini-search-high",
    "gpt-4o-search-preview": "gpt-4o-search",
    "gpt-4o-search-preview-high": "gpt-4o-search-high",
    "gpt-4o-search-preview-high-loc": "gpt-4o-search-high-loc",
    "gemini-2.0-flash-grounding": "gemini-2.0-flash-grounding",
    "gemini-2.5-flash-preview-04-17-grounding": "gemini-2.5-flash-grounding",
    "gemini-2.0-pro-exp-02-05-grounding": "gemini-2.5-pro-grounding",
    "gemini-2.5-pro-exp-03-25-grounding": "gemini-2.5-pro-grounding",
    "gemini-2.5-pro-exp-03-25-wo-search": "gemini-2.5-pro"
}

In [40]:
citations_df.sample(10)

,citation_id,response_id,citation_number,url,domain_full,domain,url_valid,citation_order
30952,cite_00030952,resp_00004749,12,https://pmc.ncbi.nlm.nih.gov/articles/PMC7440690/,pmc.ncbi.nlm.nih.gov,nih.gov,True,11
356082,cite_00356082,resp_00064127,10,https://www.kenhub.com/en/library/anatomy/live...,kenhub.com,kenhub.com,True,9
158663,cite_00158663,resp_00028197,3,https://www.simplilearn.com/generative-ai-book...,simplilearn.com,simplilearn.com,True,2
252282,cite_00252282,resp_00044040,8,https://stanmurrell.org/2021/12/16/jehovahs-wi...,stanmurrell.org,stanmurrell.org,True,7
237459,cite_00237459,resp_00041487,8,https://luatpvlgroup.com/lam-sao-de-xac-dinh-m...,luatpvlgroup.com,luatpvlgroup.com,True,7
330682,cite_00330682,resp_00059307,5,https://zju3dv.github.io/pgsr/,zju3dv.github.io,github.io,True,4
364941,cite_00364941,resp_00065595,6,https://www.bibliaon.com/estudo_biblico_maria_...,bibliaon.com,bibliaon.com,True,5
151714,cite_00151714,resp_00026961,4,https://www.ptt.cc/bbs/Hunter/M.1665911302.A.0...,ptt.cc,ptt.cc,True,3
283836,cite_00283836,resp_00050231,5,https://hostbor.com/samsung-9100-pro-ssd-review/,hostbor.com,hostbor.com,True,4
151310,cite_00151310,resp_00026888,10,https://pt.wikipedia.org/wiki/Nilo_Pe%C3%A7anha,pt.wikipedia.org,wikipedia.org,True,8


In [34]:
citations_df.domain.value_counts().head(20)

domain
youtube.com          19749
wikipedia.org        14922
reddit.com            9476
github.com            3701
google.com            3252
nih.gov               2805
reuters.com           2051
microsoft.com         1866
arxiv.org             1599
linkedin.com          1418
tiktok.com            1283
apnews.com            1225
stackoverflow.com     1149
apple.com             1012
huggingface.co         984
github.io              979
naver.com              974
csdn.net               920
wordpress.com          911
openai.com             871
Name: count, dtype: int64

In [22]:
responses_df.model_name_raw.value_counts()

model_name_raw
gpt-4o-search-preview-high-loc              6995
sonar-pro-high                              6768
sonar-reasoning-pro-high                    6455
sonar-reasoning                             5941
gemini-2.5-pro-exp-03-25-grounding          5920
gpt-4o-search-preview                       5789
gpt-4o-search-preview-high                  5747
gemini-2.0-flash-grounding                  5185
sonar                                       4513
gpt-4o-mini-search-preview                  3945
sonar-pro                                   3911
gemini-2.5-flash-preview-04-17-grounding    3359
gemini-2.5-pro-exp-03-25-wo-search          1240
Name: count, dtype: int64

In [13]:
threads_df.head()

,thread_id,original_row_id,timestamp,total_turns,winner,judge,primary_intent,secondary_intent,languages,client_country
0,thread_00000000,0,2025-03-18 16:39:00.550,1,tie,78996144fc08f820066fdaa66f170da6e158c24ad70c4e...,Other,Unassigned,[English],None
1,thread_00000001,1,2025-03-18 16:39:35.849,1,model_a,023f8e28f2b51ef62a7adb0fdf06709b8004773b41ca38...,Explanation,Unassigned,[Arabic],None
2,thread_00000002,2,2025-03-18 16:43:18.951,1,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,Info Synthesis,Factual Lookup,[German],None
3,thread_00000003,3,2025-03-18 16:43:47.861,1,tie,d3560a0fe5416204140c8b1269b3ff10108c36fe59d355...,Other,Unassigned,[English],None
4,thread_00000004,4,2025-03-18 16:47:47.787,1,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,Info Synthesis,Unassigned,"[German, English]",None


In [14]:
questions_df.head()

,question_id,thread_id,turn_number,user_query,question_role
0,q_00000000,thread_00000000,1,what models are you?,user
1,q_00000001,thread_00000001,1,تجم تفسرلي علاش فما صبع زايد في مرض blackfond ...,user
2,q_00000002,thread_00000002,1,Ellen White über die Flache Erde,user
3,q_00000003,thread_00000003,1,Test,user
4,q_00000004,thread_00000004,1,"Site: ""m.egwwritings.org/en"" über die flache Erde",user


In [15]:
responses_df.head()

,response_id,question_id,thread_id,turn_number,model_name,model_side,response_text,response_role,citation_format,llm_temperature,llm_top_p,llm_max_tokens,search_context_size,user_location_country,search_engine,scrape_engine,context_manager
0,resp_00000000,q_00000000,thread_00000000,1,sonar,a,"The term ""models"" can refer to various types o...",assistant,original,0.2,0.9,4000.0,None,None,built-in,built-in,built-in
1,resp_00000001,q_00000000,thread_00000000,1,sonar-reasoning,b,## Types of Models in the Fashion Industry \n...,assistant,original,0.2,0.9,4000.0,None,None,built-in,built-in,built-in
2,resp_00000002,q_00000001,thread_00000001,1,gpt-4o-mini-search-preview,a,مرض بلاكفان-دايموند (Diamond-Blackfan Anemia) ...,assistant,original,NaN,NaN,4000.0,medium,None,built-in,built-in,built-in
3,resp_00000003,q_00000001,thread_00000001,1,sonar-pro,b,Diamond-Blackfan anemia (DBA) is a rare geneti...,assistant,original,0.2,0.9,4000.0,None,None,built-in,built-in,built-in
4,resp_00000004,q_00000002,thread_00000002,1,sonar-reasoning,a,"Ellen White, eine bedeutende Persönlichkeit de...",assistant,original,0.2,0.9,4000.0,None,None,built-in,built-in,built-in


In [41]:
arena_raw_df.head()

,model_a,model_b,winner,judge,turn,timestamp,messages_a,messages_b,system_a_metadata,system_b_metadata,primary_intent,secondary_intent,languages
0,sonar,sonar-reasoning,tie,78996144fc08f820066fdaa66f170da6e158c24ad70c4e...,1,2025-03-18 16:39:00.550,"[{'content': 'what models are you?', 'role': '...","[{'content': 'what models are you?', 'role': '...","{'citation_format_standardized': 'original', '...","{'citation_format_standardized': 'original', '...",Other,Unassigned,[English]
1,gpt-4o-mini-search-preview,sonar-pro,model_a,023f8e28f2b51ef62a7adb0fdf06709b8004773b41ca38...,1,2025-03-18 16:39:35.849,[{'content': 'تجم تفسرلي علاش فما صبع زايد في ...,[{'content': 'تجم تفسرلي علاش فما صبع زايد في ...,"{'citation_format_standardized': 'original', '...","{'citation_format_standardized': 'original', '...",Explanation,Unassigned,[Arabic]
2,sonar-reasoning,sonar-pro,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,1,2025-03-18 16:43:18.951,[{'content': 'Ellen White über die Flache Erde...,[{'content': 'Ellen White über die Flache Erde...,"{'citation_format_standardized': 'original', '...","{'citation_format_standardized': 'original', '...",Info Synthesis,Factual Lookup,[German]
3,gpt-4o-search-preview,sonar-pro,tie,d3560a0fe5416204140c8b1269b3ff10108c36fe59d355...,1,2025-03-18 16:43:47.861,"[{'content': 'Test', 'role': 'user'}, {'conten...","[{'content': 'Test', 'role': 'user'}, {'conten...","{'citation_format_standardized': 'original', '...","{'citation_format_standardized': 'original', '...",Other,Unassigned,[English]
4,gpt-4o-mini-search-preview,sonar-pro,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,1,2025-03-18 16:47:47.787,"[{'content': 'Site: ""m.egwwritings.org/en"" übe...","[{'content': 'Site: ""m.egwwritings.org/en"" übe...","{'citation_format_standardized': 'original', '...","{'citation_format_standardized': 'original', '...",Info Synthesis,Unassigned,"[German, English]"


In [17]:
arena_raw_df.iloc[0]['system_a_metadata']['llm_config']

{'name': 'sonar',
 'params': {'max_completion_tokens': None,
  'max_output_tokens': None,
  'max_tokens': 4000.0,
  'temperature': 0.2,
  'top_p': 0.9,
  'web_search_options': None}}

In [19]:
arena_raw_df.model_a.value_counts()

model_a
gpt-4o-search-preview-high-loc              2527
sonar-pro-high                              2422
sonar-reasoning-pro-high                    2402
sonar-reasoning                             2244
gpt-4o-search-preview                       2143
gpt-4o-search-preview-high                  2099
gemini-2.5-pro-exp-03-25-grounding          2097
gemini-2.0-flash-grounding                  1881
sonar                                       1682
gpt-4o-mini-search-preview                  1478
sonar-pro                                   1469
gemini-2.5-flash-preview-04-17-grounding    1182
gemini-2.5-pro-exp-03-25-wo-search           443
Name: count, dtype: int64

In [8]:
len(model_response_mapping)

10

In [9]:
len(model_name_mapping)

10

In [10]:
model_response_mapping

{'gpt-4o-search-preview': np.int64(18531),
 'sonar-pro': np.int64(10679),
 'sonar-reasoning-pro': np.int64(6455),
 'sonar-reasoning': np.int64(5941),
 'gemini-2.0-flash': np.int64(5185),
 'gemini-2.5-pro-exp-03-25': np.int64(4879),
 'sonar': np.int64(4513),
 'gpt-4o-mini-search-preview': np.int64(3945),
 'gemini-2.5-flash-preview-04-17': np.int64(3359),
 'gemini-2.0-pro-exp-02-05': np.int64(2281)}

In [42]:
battle_data_df = pd.read_parquet("/Users/yangkc/working/llm/ai_search_arena/data/intermediate/cleaned_arena_data/leaderboard/battle_data.parquet")

In [43]:
battle_data_df

,model_a,model_b,winner,judge,timestamp,total_turns,primary_intent,secondary_intent,conv_metadata,thread_id
0,sonar,sonar-reasoning,tie,78996144fc08f820066fdaa66f170da6e158c24ad70c4e...,2025-03-18 16:39:00.550,1,Other,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00000000
1,gpt-4o-mini-search-preview,sonar-pro,model_a,023f8e28f2b51ef62a7adb0fdf06709b8004773b41ca38...,2025-03-18 16:39:35.849,1,Explanation,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00000001
2,sonar-reasoning,sonar-pro,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,2025-03-18 16:43:18.951,1,Info Synthesis,Factual Lookup,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00000002
3,gpt-4o-search-preview,sonar-pro,tie,d3560a0fe5416204140c8b1269b3ff10108c36fe59d355...,2025-03-18 16:43:47.861,1,Other,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00000003
4,gpt-4o-mini-search-preview,sonar-pro,model_b,febd105f4f29cf47f72d9b5bf825bdb848fb009b7b0cd6...,2025-03-18 16:47:47.787,1,Info Synthesis,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00000004
...,...,...,...,...,...,...,...,...,...,...
24064,gpt-4o-search-preview,gemini-2.5-flash-preview-04-17,model_b,43956094e8aa8b172f5d869d2b155e6dd7a86065fd9e1f...,2025-05-09 05:10:41.078,3,Guidance,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00024064
24065,gpt-4o-search-preview,sonar-pro,model_b,1eda0ecd002074b1624f41b7bea304ad5f8166575705e4...,2025-05-09 05:16:52.273,1,Factual Lookup,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00024065
24066,gpt-4o-search-preview,sonar-pro,None,eb8e298d3bf023a06c2bc40fbf35b54304d45eac20bfe9...,2025-05-09 05:16:59.602,6,Explanation,Unassigned,"{'cites_academic_journal_a': 1.0, 'cites_acade...",thread_00024066
24067,gpt-4o-search-preview,gemini-2.5-flash-preview-04-17,None,1a1ce960efb853bcf8a7a712d0b4777460b9fcd53a1ada...,2025-05-09 05:17:15.568,3,Recommendation,Unassigned,"{'cites_academic_journal_a': 0.0, 'cites_acade...",thread_00024067


# Citation analysis

In [59]:
citations_df = pd.read_parquet(cleaned_arena_data_root / "citations.parquet")
citations_enriched_df = pd.read_parquet(cleaned_arena_data_root / "citations_enriched.parquet")
domain_list_df = pd.read_parquet(cleaned_arena_data_root / "domains.parquet")
domain_enriched_df = pd.read_parquet(cleaned_arena_data_root / "domains_enriched.parquet")

In [60]:
domain_enriched_df.political_leaning.value_counts()

political_leaning
unknown          70515
left_leaning     10589
right_leaning     2429
Name: count, dtype: int64

In [61]:
domain_enriched_df.domain_quality.value_counts()

domain_quality
unknown         82010
low_quality       772
high_quality      751
Name: count, dtype: int64

In [63]:
domain_enriched_df.query("domain_quality == 'unknown'").head(20)

,domain,citation_count,political_leaning_score,political_leaning,domain_quality_score,domain_quality
2,reddit.com,9476,-0.193405,left_leaning,NaN,unknown
3,github.com,3701,-0.351924,left_leaning,NaN,unknown
7,microsoft.com,1866,-0.148023,left_leaning,NaN,unknown
8,arxiv.org,1599,-0.428459,left_leaning,NaN,unknown
9,linkedin.com,1418,-0.099147,left_leaning,NaN,unknown
10,tiktok.com,1283,-0.185440,left_leaning,NaN,unknown
12,stackoverflow.com,1149,-0.329961,left_leaning,NaN,unknown
14,huggingface.co,984,-0.573013,left_leaning,NaN,unknown
15,github.io,979,-0.430949,left_leaning,NaN,unknown
16,naver.com,974,-0.304714,left_leaning,NaN,unknown


In [64]:
domain_frequency_df = citations_df.domain.value_counts().to_frame(name='freq').reset_index()

In [66]:
citations_df

,citation_id,response_id,citation_number,url,domain_full,domain,url_valid,citation_order
0,cite_00000000,resp_00000000,1,https://www.pixpa.com/blog/types-of-models,pixpa.com,pixpa.com,True,0
1,cite_00000001,resp_00000000,2,https://brainfall.com/quizzes/which-supermodel...,brainfall.com,brainfall.com,True,1
2,cite_00000002,resp_00000000,3,https://uquiz.com/quiz/CsHn01/which-supermodel...,uquiz.com,uquiz.com,True,2
3,cite_00000003,resp_00000000,4,https://www.quotev.com/quizzes/Which+Model+Are...,quotev.com,quotev.com,True,3
4,cite_00000004,resp_00000000,5,https://www.quotev.com/quizzes/Which+Model+Are...,quotev.com,quotev.com,True,4
...,...,...,...,...,...,...,...,...
366082,cite_00366082,resp_00065766,3,https://wellplayed.media/column-yukakuramoti-c...,wellplayed.media,wellplayed.media,True,2
366083,cite_00366083,resp_00065766,4,https://www.4gamer.net/games/397/G039789/20191...,4gamer.net,4gamer.net,True,3
366084,cite_00366084,resp_00065766,5,https://fgamers.saikyou.biz/?ときど,fgamers.saikyou.biz,saikyou.biz,True,4
366085,cite_00366085,resp_00065766,6,https://global.honda/jp/guide/history-digest/7...,global.honda,global.honda,True,5


In [68]:
citations_df.query("domain == 'youtube.com'")

14082

In [65]:
domain_frequency_df.to_csv("domain_frequency.csv", index=None)

In [48]:
citations_enriched_df.columns

Index(['citation_id', 'response_id', 'citation_number', 'url', 'domain_full',
       'domain', 'url_valid', 'citation_order', 'leaning_score_users',
       'is_left_leaning', 'domain_quality'],
      dtype='object')

In [49]:
citations_enriched_df

,citation_id,response_id,citation_number,url,domain_full,domain,url_valid,citation_order,leaning_score_users,is_left_leaning,domain_quality
0,cite_00000000,resp_00000000,1,https://www.pixpa.com/blog/types-of-models,pixpa.com,pixpa.com,True,0,NaN,None,NaN
1,cite_00000001,resp_00000000,2,https://brainfall.com/quizzes/which-supermodel...,brainfall.com,brainfall.com,True,1,-0.057987,True,NaN
2,cite_00000002,resp_00000000,3,https://uquiz.com/quiz/CsHn01/which-supermodel...,uquiz.com,uquiz.com,True,2,-0.319297,True,NaN
3,cite_00000003,resp_00000000,4,https://www.quotev.com/quizzes/Which+Model+Are...,quotev.com,quotev.com,True,3,-0.086263,True,NaN
4,cite_00000004,resp_00000000,5,https://www.quotev.com/quizzes/Which+Model+Are...,quotev.com,quotev.com,True,4,-0.086263,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...
366082,cite_00366082,resp_00065766,3,https://wellplayed.media/column-yukakuramoti-c...,wellplayed.media,wellplayed.media,True,2,NaN,None,NaN
366083,cite_00366083,resp_00065766,4,https://www.4gamer.net/games/397/G039789/20191...,4gamer.net,4gamer.net,True,3,-0.421465,True,NaN
366084,cite_00366084,resp_00065766,5,https://fgamers.saikyou.biz/?ときど,fgamers.saikyou.biz,saikyou.biz,True,4,NaN,None,NaN
366085,cite_00366085,resp_00065766,6,https://global.honda/jp/guide/history-digest/7...,global.honda,global.honda,True,5,-0.128852,True,NaN
